# Deploy App to Dashboard

Example of how to deploy a langchain app into the streamlit dashboard.

In [1]:
# trulens_eval notebook dev

%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

base = Path().cwd().parent.parent.parent / "trulens_eval"

# If running from github repo, can use this:
sys.path.append(str(base))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

from trulens_eval.keys import check_keys

check_keys(
    "OPENAI_API_KEY",
    "HUGGINGFACE_API_KEY"
)

from trulens_eval import Tru
tru = Tru()
tru.reset_database()

tru.run_dashboard(_dev=base, force=True)

✅ Key OPENAI_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).
✅ Key HUGGINGFACE_API_KEY set from environment (same value found in .env file at /Users/piotrm/Dropbox/repos/github/trulens/.env).
🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.
Deleted 2 rows.
Force stopping dashboard ...
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.86.113:8501 .


In [2]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import TruChain, Feedback, Huggingface, Tru
tru = Tru()

# Imports from langchain to build app. You may need to install langchain first
# with the following:
# ! pip install langchain>=0.0.170
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts.chat import ChatPromptTemplate, PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate

In [3]:
def load_app():
    full_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=
            "Provide a helpful response with relevant background information for the following: {prompt}",
            input_variables=["prompt"],
        )
    )

    chat_prompt_template = ChatPromptTemplate.from_messages([full_prompt])

    llm = OpenAI(temperature=0.9, max_tokens=128)

    chain = LLMChain(llm=llm, prompt=chat_prompt_template, verbose=True)

    return chain

chain = load_app()

In [4]:
# prompt_input = '¿que hora es?'
# llm_response = chain(prompt_input)
# display(llm_response)

In [5]:
# Initialize Huggingface-based feedback function collection class:
# hugs = Huggingface()

# Define a language match feedback function using HuggingFace.
# f_lang_match = Feedback(hugs.language_match).on_input_output()
# By default this will check language match on the main app input and main app
# output.

In [7]:
tru_recorder = TruChain(chain,
    app_id='Chain1_ChatApplication',
    initial_app_loader = load_app
#    feedbacks=[f_lang_match]
)

Wrote loadable app to /Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/examples/quickstart/Chain1_ChatApplication.json and /Users/piotrm/Dropbox/repos/github/trulens/trulens_eval/examples/quickstart/Chain1_ChatApplication.dill.


In [ ]:
from trulens_eval.utils.json import jsonify
from trulens_eval.schema import AppDefinition

In [ ]:
list(tru_recorder.dict().keys())

In [ ]:
app_json = jsonify(tru_recorder)

In [ ]:
s = AppDefinition.new_session(app_json)

In [ ]:
with s as rec:
    s.app("hello there")
print(rec.get())

In [ ]:
with tru_recorder as recording:
    llm_response = chain(prompt_input)

display(llm_response)

In [ ]:
# tru.run_dashboard() # open a local streamlit app to explore

# tru.stop_dashboard() # stop if needed